In [28]:
import pandas as pd
import numpy as np
import pickle

from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Flatten
from keras.utils import np_utils
from keras.preprocessing.text import one_hot
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing

In [2]:
train_df = pd.read_csv('D:/Program/dataset/Spooky_Author_Identification/train.csv')
test_df = pd.read_csv('D:/Program/dataset/Spooky_Author_Identification/test.csv')

# Label Onehot encoding  

le = LabelEncoder()
le.fit(train_df['author'])
print(le.classes_)

y = le.transform(train_df['author'])
y_train = np_utils.to_categorical(y)

y_train

['EAP' 'HPL' 'MWS']


array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.]])

In [3]:
with open('keras_input_train.pkl', 'rb') as f:
    sequences = pickle.load(f)
with open('keras_input_test.pkl', 'rb') as f:
    test_sequences = pickle.load(f)
    
x_train = sequences
x_test = test_sequences

print(x_train.shape)
print(x_test.shape)

(19579, 24992)
(8392, 24992)


In [15]:
# TIME_STEPS = 28     # same as the height of the image
# INPUT_SIZE = 28     # same as the width of the image
# BATCH_SIZE = 50
# BATCH_INDEX = 0
# OUTPUT_SIZE = 10
# CELL_SIZE = 100

NUM_WORDS = 200
N = 20
MAX_LEN = x_train.shape[1]


In [22]:
# model = Sequential() 
# model.add(Embedding(NUM_WORDS, N, input_length=500))
# model.add(LSTM(N, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
# model.add(Flatten())
# # model.add(Dense(64, activation='relu'))
# # model.add(Dropout(0.2))
# model.add(Dense(3, activation='softmax',name="output"))
# model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 20)           4000      
_________________________________________________________________
lstm_6 (LSTM)                (None, 500, 20)           3280      
_________________________________________________________________
flatten_5 (Flatten)          (None, 10000)             0         
_________________________________________________________________
output (Dense)               (None, 3)                 30003     
Total params: 37,283
Trainable params: 37,283
Non-trainable params: 0
_________________________________________________________________


In [23]:
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
# train_history = model.fit(x_train.todense(), y_train, validation_split=0.2, epochs=5, batch_size=20, verbose=1)

In [29]:
NUM_WORDS = 16000
N = 12
MAX_LEN = 300

tmp_X = train_df['text']
tmp_Y = train_df['author']
tmp_X_test = test_df['text']

tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(tmp_X)

ttrain_x = tokenizer.texts_to_sequences(tmp_X)
ttrain_x = pad_sequences(ttrain_x, maxlen=MAX_LEN)

ttest_x = tokenizer.texts_to_sequences(tmp_X_test)
ttest_x = pad_sequences(ttest_x, maxlen=MAX_LEN)

lb = preprocessing.LabelBinarizer()
lb.fit(tmp_Y)

ttrain_y = lb.transform(tmp_Y)

In [31]:
model = Sequential()
model.add(Embedding(NUM_WORDS, N, input_length=MAX_LEN))
model.add(LSTM(N, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 300, 12)           192000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 300, 12)           1200      
_________________________________________________________________
flatten_7 (Flatten)          (None, 3600)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                230464    
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 195       
Total params: 423,859
Trainable params: 423,859
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.fit(ttrain_x, ttrain_y, 
                  validation_split=0.1,
                  batch_size=128, epochs=10, 
                  verbose=1,
                 )

Train on 17621 samples, validate on 1958 samples
Epoch 1/10
17621/17621 [==============================] - 65s - loss: 0.9772 - acc: 0.5081 - val_loss: 0.6951 - val_acc: 0.7217
Epoch 2/10
17621/17621 [==============================] - 61s - loss: 0.4926 - acc: 0.8061 - val_loss: 0.4365 - val_acc: 0.8258
Epoch 3/10
17621/17621 [==============================] - 61s - loss: 0.2853 - acc: 0.8949 - val_loss: 0.4143 - val_acc: 0.8407
Epoch 4/10
17621/17621 [==============================] - 61s - loss: 0.1896 - acc: 0.9325 - val_loss: 0.4599 - val_acc: 0.8447
Epoch 5/10
17621/17621 [==============================] - 62s - loss: 0.1376 - acc: 0.9528 - val_loss: 0.5009 - val_acc: 0.8304
Epoch 6/10
17621/17621 [==============================] - 61s - loss: 0.1080 - acc: 0.9642 - val_loss: 0.5504 - val_acc: 0.8330
Epoch 7/10
17621/17621 [==============================] - 61s - loss: 0.0817 - acc: 0.9720 - val_loss: 0.6437 - val_acc: 0.8299
Epoch 8/10
17621/17621 [==============================]

In [33]:
scores = model.evaluate(ttrain_x, ttrain_y)  
print()
print(scores)

19579/19579 [==============================] - 64s    

[0.098476597449195569, 0.97701619085626745]


In [34]:
%%time
prediction = model.predict(ttest_x)
print()
print("Show 10 prediction result :")  
print(prediction[:10])
print(prediction.shape)


Show 10 prediction result :
[[  3.33677488e-03   4.13969217e-04   9.96249259e-01]
 [  9.99999881e-01   8.78515110e-08   7.32659888e-09]
 [  1.34132293e-04   9.99862909e-01   3.00555143e-06]
 [  9.99899030e-01   1.00462166e-04   5.33828427e-07]
 [  9.98400390e-01   9.35346645e-04   6.64288469e-04]
 [  9.98546362e-01   1.42261619e-03   3.11006479e-05]
 [  9.93996382e-01   4.40580165e-03   1.59775652e-03]
 [  5.40444031e-02   2.11553052e-02   9.24800277e-01]
 [  1.00000000e+00   1.14381160e-09   3.30956755e-11]
 [  9.96955276e-01   8.84667097e-04   2.16001016e-03]]
(8392, 3)
Wall time: 25.8 s


In [35]:
# submission

submission = pd.DataFrame(prediction, columns=le.classes_)
submission["id"] = test_df["id"]
cols = submission.columns.tolist()
cols = cols[-1:]+cols[:-1]
submission = submission[cols]
submission.head(10)

,id,EAP,HPL,MWS
0,id02310,0.003337,4.139692e-04,9.962493e-01
1,id24541,1.000000,8.785151e-08,7.326599e-09
2,id00134,0.000134,9.998629e-01,3.005551e-06
3,id27757,0.999899,1.004622e-04,5.338284e-07
4,id04081,0.998400,9.353466e-04,6.642885e-04
5,id27337,0.998546,1.422616e-03,3.110065e-05
6,id24265,0.993996,4.405802e-03,1.597757e-03
7,id25917,0.054044,2.115531e-02,9.248003e-01
8,id04951,1.000000,1.143812e-09,3.309568e-11
9,id14549,0.996955,8.846671e-04,2.160010e-03


In [36]:
submission.to_csv('submission_lstm.csv',index=False)